In [0]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.functions import *

#-----------------------------------#

sc = SparkContext.getOrCreate()
spark =  SparkSession(sc)


In [0]:
from pyspark.sql.types import StructType,StructField,StringType

schema = StructType([
    StructField("language", StringType(), True),
    StructField("year", StringType(), True),
    StructField("users", StringType(), True),
  StructField("admin", StringType(), True),
  StructField("country", StringType(), True)
  
])

data  = (('java','2020','222222','somesh','india'),('python','2222','222222','ram','usa'),('Scala','2021','222222','sol','germany'))

df  = spark.createDataFrame(data,schema)
df.show()

+--------+----+------+------+-------+
language|year| users| admin|country|
+--------+----+------+------+-------+
 java|2020|222222|somesh| india|
 python|2222|222222| ram| usa|
 Scala|2021|222222| sol|germany|
+--------+----+------+------+-------+

In [0]:
cols = ['admin','country']
df= df.drop(*cols)
df.show()

+--------+----+------+
language|year| users|
+--------+----+------+
 java|2020|222222|
 python|2222|222222|
 Scala|2021|222222|
+--------+----+------+

In [0]:
df2 = df.withColumn("year", when(col("year") == "2020","PAST")
      .when(col("year") == "2222","Future")
      .otherwise("Current"))

In [0]:
df2.show()

+--------+-------+------+
language| year| users|
+--------+-------+------+
 java| PAST|222222|
 python| Future|222222|
 Scala|Current|222222|
+--------+-------+------+

In [0]:
df3 = df.select(col("*"),when(col("year") == "2020","PAST")
      .when(col("year") == "2222","Future")
      .otherwise("Current").alias("New"))

df3.show()


+--------+----+------+-------+
language|year| users| New|
+--------+----+------+-------+
 java|2020|222222| PAST|
 python|2222|222222| Future|
 Scala|2021|222222|Current|
+--------+----+------+-------+

In [0]:
df1 = df.withColumn('new_col',expr("case when year = '2020' then 'PAST'"+
                              "when year = 2021 then 'Current'"+
                                  "else 'Future' end"))

df1.show()

+--------+----+------+-------+
language|year| users|new_col|
+--------+----+------+-------+
 java|2020|222222| PAST|
 python|2222|222222| Future|
 Scala|2021|222222|Current|
+--------+----+------+-------+

In [0]:
df4 = df.select(col("*"),expr("case when year = '2020' then 'PAST'"+
                              "when year = 2021 then 'Current'"+
                                  "else 'Future' end").alias('new_col'))

df4.show()

+--------+----+------+-------+
language|year| users|new_col|
+--------+----+------+-------+
 java|2020|222222| PAST|
 python|2222|222222| Future|
 Scala|2021|222222|Current|
+--------+----+------+-------+

In [0]:
from pyspark.sql.types import StructType,StructField,StringType,IntegerType
data= ((66, "a", "4"), (67, "a", "0"), (70, "b", "4"), (71, "d", "4"))



schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("code", StringType(), True),
    StructField("amt", StringType(), True)])
dataDF = spark.createDataFrame(data,schema)
dataDF.show()

+---+----+---+
 id|code|amt|
+---+----+---+
 66| a| 4|
 67| a| 0|
 70| b| 4|
 71| d| 4|
+---+----+---+

In [0]:
ddf = dataDF.withColumn("new_column",
       when((col("code") == "a") | (col("code") == "d"), "A")
      .when((col("code") == "b") & (col("amt") == "4"), "B")
      .otherwise("A1"))


ddf.show()

+---+----+---+----------+
 id|code|amt|new_column|
+---+----+---+----------+
 66| a| 4| A|
 67| a| 0| A|
 70| b| 4| B|
 71| d| 4| A|
+---+----+---+----------+

In [0]:
from pyspark.sql import Row
l = [('Alice',2),('somesh',4),('soh',54),('raja',6),('bob',8),('cate',9)]
Person = Row('name','age')
rdd = sc.parallelize(l)
person = rdd.map(lambda r:Person(*r))
df2 = sqlContext.createDataFrame(person)
df2.show()

+------+---+
 name|age|
+------+---+
 Alice| 2|
somesh| 4|
 soh| 54|
 raja| 6|
 bob| 8|
 cate| 9|
+------+---+

In [0]:
from pyspark.sql.types import * 

rdd = sc.parallelize(l)
schema =  StructType([StructField ("name" , StringType(), True) , 
StructField("age" , IntegerType(), True)]) 
df3 = sqlContext.createDataFrame(rdd, schema) 
df3.show()

+------+---+
 name|age|
+------+---+
 Alice| 2|
somesh| 4|
 soh| 54|
 raja| 6|
 bob| 8|
 cate| 9|
+------+---+

In [0]:
df = df3.select(col("name"),col("age"),lit("1").alias("literals"))
df.show()

+------+---+--------+
 name|age|literals|
+------+---+--------+
 Alice| 2| 1|
somesh| 4| 1|
 soh| 54| 1|
 raja| 6| 1|
 bob| 8| 1|
 cate| 9| 1|
+------+---+--------+

In [0]:
df = df3.withColumn("lit_value2",
    when((col("Age") >=2 )&(col("age") <= 8), 
       lit("8"))
      .otherwise(lit("9")))
df.show()

+------+---+----------+
 name|age|lit_value2|
+------+---+----------+
 Alice| 2| 8|
somesh| 4| 8|
 soh| 54| 9|
 raja| 6| 8|
 bob| 8| 8|
 cate| 9| 9|
+------+---+----------+

In [0]:
StringType	ShortType
ArrayType	IntegerType
MapType	LongType
StructType	FloatType
DateType	DoubleType
TimestampType	DecimalType
BooleanType	ByteType
CalendarIntervalType	HiveStringType
BinaryType	ObjectType
NumericType	NullType

In [0]:
simpleData = (("James",34,"2006-01-01","true","M",3000.60),
    ("Michael",33,"1980-01-10","true","F",3300.80),
    ("Robert",37,"06-01-1992","false","M",5000.50)
  )

simpleSchema = StructType((
    StructField("firstName",StringType(),True),
    StructField("age",IntegerType(),True),
    StructField("jobStartDate",StringType(),True),
    StructField("isGraduated", StringType(), True),
    StructField("gender", StringType(), True),
    StructField("salary", DoubleType(), True)
  ))

df = spark.createDataFrame(data = simpleData,schema = simpleSchema)
df.printSchema()
df.show()

root
-- firstName: string (nullable = true)
-- age: integer (nullable = true)
-- jobStartDate: string (nullable = true)
-- isGraduated: string (nullable = true)
-- gender: string (nullable = true)
-- salary: double (nullable = true)

+---------+---+------------+-----------+------+------+
firstName|age|jobStartDate|isGraduated|gender|salary|
+---------+---+------------+-----------+------+------+
 James| 34| 2006-01-01| true| M|3000.6|
 Michael| 33| 1980-01-10| true| F|3300.8|
 Robert| 37| 06-01-1992| false| M|5000.5|
+---------+---+------------+-----------+------+------+

In [0]:
df2 = df.withColumn("age",col("age").cast(StringType())).withColumn("isGraduated",col("isGraduated").cast(BooleanType())) .withColumn("jobStartDate",col("jobStartDate").cast(DateType()))
df2.printSchema()

root
-- firstName: string (nullable = true)
-- age: string (nullable = true)
-- jobStartDate: date (nullable = true)
-- isGraduated: boolean (nullable = true)
-- gender: string (nullable = true)
-- salary: double (nullable = true)

In [0]:
df3 = df2.selectExpr("cast(age as int) age",
    "cast(isGraduated as string) isGraduated",
    "cast(jobStartDate as string) jobStartDate")
df3.printSchema()
df3.show()

root
-- age: integer (nullable = true)
-- isGraduated: string (nullable = true)
-- jobStartDate: string (nullable = true)

+---+-----------+------------+
age|isGraduated|jobStartDate|
+---+-----------+------------+
 34| true| 2006-01-01|
 33| true| 1980-01-10|
 37| false| null|
+---+-----------+------------+

In [0]:
df3.createOrReplaceTempView("CastExample")
df4 = spark.sql("SELECT STRING(age),BOOLEAN(isGraduated), \
        DATE(jobStartDate) from CastExample")
df4.printSchema()
df4.show()

root
-- age: string (nullable = true)
-- isGraduated: boolean (nullable = true)
-- jobStartDate: date (nullable = true)

+---+-----------+------------+
age|isGraduated|jobStartDate|
+---+-----------+------------+
 34| true| 2006-01-01|
 33| true| 1980-01-10|
 37| false| null|
+---+-----------+------------+

In [0]:
data = (("Banana",1000,"USA"), ("Carrots",1500,"USA"), ("Beans",1600,"USA"),
      ("Orange",2000,"USA"),("Orange",2000,"USA"),("Banana",400,"China"),
      ("Carrots",1200,"China"),("Beans",1500,"China"),("Orange",4000,"China"),
      ("Banana",2000,"Canada"),("Carrots",2000,"Canada"),("Beans",2000,"Mexico"))

simpleSchema = StructType((
    StructField("Product",StringType(),True),
    StructField("Amount",IntegerType(),True),
    StructField("Country",StringType(),True)))

df = spark.createDataFrame(data,simpleSchema)
df.show()

+-------+------+-------+
Product|Amount|Country|
+-------+------+-------+
 Banana| 1000| USA|
Carrots| 1500| USA|
 Beans| 1600| USA|
 Orange| 2000| USA|
 Orange| 2000| USA|
 Banana| 400| China|
Carrots| 1200| China|
 Beans| 1500| China|
 Orange| 4000| China|
 Banana| 2000| Canada|
Carrots| 2000| Canada|
 Beans| 2000| Mexico|
+-------+------+-------+

In [0]:
pivotDF = df.groupBy("Product").pivot("Country").sum("Amount")
pivotDF.show()

+-------+------+-----+------+----+
Product|Canada|China|Mexico| USA|
+-------+------+-----+------+----+
 Orange| null| 4000| null|4000|
 Beans| null| 1500| 2000|1600|
 Banana| 2000| 400| null|1000|
Carrots| 2000| 1200| null|1500|
+-------+------+-----+------+----+

In [0]:
unPivotDF = pivotDF.select("Product",
expr("stack(3, 'Canada', Canada, 'China', China, 'Mexico', Mexico) as (Country,Total)")) \
.where("Total is not null")
unPivotDF.show()

+-------+-------+-----+
Product|Country|Total|
+-------+-------+-----+
 Orange| China| 4000|
 Beans| China| 1500|
 Beans| Mexico| 2000|
 Banana| Canada| 2000|
 Banana| China| 400|
Carrots| Canada| 2000|
Carrots| China| 1200|
+-------+-------+-----+

In [0]:
simpleData = (("James","Sales",3000),
      ("Michael","Sales",4600),
      ("Robert","Sales",4100),
      ("Maria","Finance",3000),
      ("Raman","Finance",3000),
      ("Scott","Finance",3300),
      ("Jen","Finance",3900),
      ("Jeff","Marketing",3000),
      ("Kumar","Marketing",2000)
    )

simpleSchema = StructType((
    StructField("Name",StringType(),True),
    StructField("Dept",StringType(),True),
    StructField("Salary",StringType(),True)))

df = spark.createDataFrame(data = simpleData,schema  = simpleSchema)
df.show()

+-------+---------+------+
 Name| Dept|Salary|
+-------+---------+------+
 James| Sales| 3000|
Michael| Sales| 4600|
 Robert| Sales| 4100|
 Maria| Finance| 3000|
 Raman| Finance| 3000|
 Scott| Finance| 3300|
 Jen| Finance| 3900|
 Jeff|Marketing| 3000|
 Kumar|Marketing| 2000|
+-------+---------+------+

In [0]:
from pyspark.sql import Window
w2 = Window.partitionBy("dept").orderBy(col("salary"))
df.withColumn("row",row_number().over(w2)) \
      .where(col("row") == '1').drop("row") \
      .show()

+-----+---------+------+
 Name| Dept|Salary|
+-----+---------+------+
Maria| Finance| 3000|
Kumar|Marketing| 2000|
James| Sales| 3000|
+-----+---------+------+

In [0]:
w3 = Window.partitionBy("dept").orderBy(col("salary").desc())
df.withColumn("row",row_number().over(w3)) \
      .where(col("row") == '1').drop("row").show()

+-------+---------+------+
 Name| Dept|Salary|
+-------+---------+------+
 Jen| Finance| 3900|
 Jeff|Marketing| 3000|
Michael| Sales| 4600|
+-------+---------+------+

In [0]:
w4 = Window.partitionBy("dept")
aggDF = df.withColumn("row",row_number().over(w3)).withColumn("avg", avg(col("salary")).over(w4)).withColumn("sum", sum(col("salary")).over(w4)).withColumn("min", min(col("salary")).over(w4)).withColumn("max", max(col("salary")).over(w4)) .where(col("row")==1).select("dept","avg","sum","min","max").show()

+---------+------+-------+----+----+
 dept| avg| sum| min| max|
+---------+------+-------+----+----+
 Finance|3300.0|13200.0|3000|3900|
Marketing|2500.0| 5000.0|2000|3000|
 Sales|3900.0|11700.0|3000|4600|
+---------+------+-------+----+----+